In [2]:
import numpy as np
import time

# Scale up the board size for larger gameplay
GRID_SIZE = 35  

SAFE_ZONE = -1   # 🔲 Safe Zones (Edges)
CIVILIAN = 1     # 👥 Civilians needing rescue
DISASTER = 2     # 🔥 Fire spreading
EMS_VEHICLE = 3  # 🚑 EMS Vehicles
FIREFIGHTER = 6  # 🚒 Firefighters
ROAD = 4         # 🛣️ Roads (1-block-wide & 3-block-wide)
BUILDING = 5     # 🏠 Houses (4-block clusters)
WATER = 7        # 🔵 Fire being extinguished (blue phase)
EXTINGUISHED = 8 # ⬜ Fire completely out (grey phase)

# Define emoji map for visualization
emoji_map = {
    SAFE_ZONE: "🔲",  
    0: "🟩",    # Grass
    CIVILIAN: "👥",
    DISASTER: "🔥",
    EMS_VEHICLE: "🚑",
    FIREFIGHTER: "🚒",
    ROAD: "⬜",    # Light Grey Roads
    BUILDING: "🏠", # Houses (Expanded to 4 blocks)
    WATER: "🔵",   # Firefighters spraying water
    EXTINGUISHED: "⬜"  # Fire completely put out
}

# Key to be printed next to the board
KEY_DESCRIPTION = {
    "🔲": "Safe Zone (Edge)",
    "🟩": "Grass",
    "👥": "Civilians needing rescue",
    "🔥": "Fire (spreading)",
    "🚑": "EMS Vehicle",
    "🚒": "Firefighter",
    "⬜": "Road (1-block & 3-block)",
    "🏠": "Houses",
    "🔵": "Fire being extinguished",
    "⬜": "Fire fully out"
}

# Function to display board with key
def show_grid_with_key(grid, step):
    if step % 5 == 0:  
        grid_rows = ["".join(emoji_map[cell] for cell in row) for row in grid]
        key_rows = [f"{symbol} - {desc}" for symbol, desc in KEY_DESCRIPTION.items()]
        
        max_rows = max(len(grid_rows), len(key_rows))
        grid_rows += [""] * (max_rows - len(grid_rows))
        key_rows += [""] * (max_rows - len(key_rows))

        for grid_row, key_row in zip(grid_rows, key_rows):
            print(f"{grid_row}   |   {key_row}")
        print("\n")

# Initialize grid with grass
grid = np.zeros((GRID_SIZE, GRID_SIZE))

# Set up safe zones (outer edges)
grid[0, :] = SAFE_ZONE
grid[-1, :] = SAFE_ZONE
grid[:, 0] = SAFE_ZONE
grid[:, -1] = SAFE_ZONE

# Create a cross-shaped road network (3 blocks wide)
mid = GRID_SIZE // 2
road_width = 1  

for i in range(-road_width, road_width + 1):
    grid[mid + i, :] = ROAD  
    grid[:, mid + i] = ROAD  

# Place houses as 4-block clusters
num_houses = 40  
houses = []
for _ in range(num_houses):
    x, y = np.random.randint(1, GRID_SIZE-3, size=2)  
    if np.all(grid[x:x+2, y:y+2] == 0):  
        grid[x:x+2, y:y+2] = BUILDING  
        houses.append((x, y))

# Connect each house to the nearest road
def connect_house_to_road(grid, house_x, house_y):
    road_positions = np.argwhere(grid == ROAD)  
    best_road = min(road_positions, key=lambda r: abs(r[0] - house_x) + abs(r[1] - house_y))
    cx, cy = house_x, house_y

    while (cx, cy) != tuple(best_road):
        grid[cx, cy] = ROAD  
        if abs(cx - best_road[0]) > abs(cy - best_road[1]):
            cx += np.sign(best_road[0] - cx)
        else:
            cy += np.sign(best_road[1] - cy)

for hx, hy in houses:
    connect_house_to_road(grid, hx, hy)

# Place civilians near houses
num_civilians = 15  
civilians = []
for _ in range(num_civilians):
    house_x, house_y = houses[np.random.randint(len(houses))]
    offsets = [(-1, 0), (1, 0), (0, -1), (0, 1)]
    np.random.shuffle(offsets)  
    for dx, dy in offsets:
        cx, cy = house_x + dx, house_y + dy
        if 1 <= cx < GRID_SIZE-1 and 1 <= cy < GRID_SIZE-1 and grid[cx, cy] == 0:
            grid[cx, cy] = CIVILIAN
            civilians.append((cx, cy))
            break

# Place EMS vehicles on roads
ems_units = [(mid-1, mid), (mid+1, mid)]  
for ems_x, ems_y in ems_units:
    grid[ems_x, ems_y] = EMS_VEHICLE

# Increase the number of fire trucks
firefighters = [(mid-3, mid+1), (mid+3, mid-1), (mid-2, mid-2), (mid+2, mid+2)]  
for fx, fy in firefighters:
    grid[fx, fy] = FIREFIGHTER

# Place a fire randomly
disaster_x, disaster_y = np.random.randint(1, GRID_SIZE-1, size=2)
grid[disaster_x, disaster_y] = DISASTER

# Fire spreading logic
def spread_fire(grid, step):
    new_grid = grid.copy()
    spread_chance = min(0.1 + (step * 0.05), 0.6)  
    for x in range(1, GRID_SIZE-1):
        for y in range(1, GRID_SIZE-1):
            if grid[x, y] == DISASTER:
                for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                    if np.random.rand() < spread_chance and grid[x+dx, y+dy] in [0, ROAD, BUILDING]:  
                        new_grid[x+dx, y+dy] = DISASTER
    return new_grid

# Fire extinguishing logic
def firefighters_extinguish(grid, firefighters):
    for fx, fy in firefighters:
        for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            nx, ny = fx + dx, fy + dy
            if 0 <= nx < GRID_SIZE and 0 <= ny < GRID_SIZE and grid[nx, ny] == DISASTER:
                grid[nx, ny] = WATER  
    return grid

# Removing fire after extinguishing
def remove_fire(grid):
    for x in range(GRID_SIZE):
        for y in range(GRID_SIZE):
            if grid[x, y] == WATER:
                grid[x, y] = EXTINGUISHED  
            elif grid[x, y] == EXTINGUISHED:
                grid[x, y] = 0  
    return grid

# Start timer for tracking rescue time
rescued_count = 0
max_rescues = 15
total_evacuated_time = 0  

# Run simulation
steps = 25  
for step in range(steps):
    if rescued_count >= max_rescues:
        break
    grid = spread_fire(grid, step)
    grid = firefighters_extinguish(grid, firefighters)
    grid = remove_fire(grid)
    show_grid_with_key(grid, step)
    time.sleep(0.5)

print(f"\n✅ Simulation complete! {rescued_count} civilians rescued in {total_evacuated_time} minutes. ⏳🚑")

🔲🔲🔲🔲🔲🔲🔲🔲🔲🔲🔲🔲🔲🔲🔲🔲⬜⬜⬜🔲🔲🔲🔲🔲🔲🔲🔲🔲🔲🔲🔲🔲🔲🔲🔲   |   🔲 - Safe Zone (Edge)
🔲🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩⬜⬜⬜🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🔲   |   🟩 - Grass
🔲🟩🟩⬜⬜⬜🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩⬜⬜⬜🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🔲   |   👥 - Civilians needing rescue
🔲🟩🟩🏠🏠⬜⬜🟩🟩🟩🟩🟩🟩🟩🟩🟩⬜⬜⬜🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🔲   |   🔥 - Fire (spreading)
🔲🟩🟩🟩🟩🟩⬜⬜🟩🟩🟩🟩🟩🟩🟩🟩⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜🏠🟩🟩🟩🟩🟩🔲   |   🚑 - EMS Vehicle
🔲🟩🟩🟩🟩🟩👥⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜🟩🟩🟩🟩👥⬜🏠🟩🏠🏠🟩🟩🟩🟩🟩🔲   |   🚒 - Firefighter
🔲🟩🟩🟩🟩🟩🟩🏠🏠🟩🟩🟩⬜🟩🟩🟩⬜⬜⬜🟩👥🟩🟩🟩🏠🏠🟩🟩🟩🟩🟩🟩🟩🟩🔲   |   ⬜ - Fire fully out
🔲🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩⬜🟩🟩🟩⬜⬜⬜⬜⬜🏠🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🔲   |   🏠 - Houses
🔲🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩⬜🟩🟩🟩⬜⬜⬜🟩🏠🏠🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🔲   |   🔵 - Fire being extinguished
🔲🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩👥⬜🏠🟩🟩⬜⬜⬜🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🔲   |   
🔲🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🏠🏠🟩🟩⬜⬜⬜🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🔲   |   
🔲👥👥⬜🏠🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩⬜⬜⬜🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🔲   |   
🔲⬜⬜⬜🏠🟩🟩🟩🟩🟩⬜🏠🟩🟩🟩🟩⬜⬜⬜🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🔲   |   
🔲🏠🏠⬜🟩🟩🟩🟩🟩🟩⬜🏠🟩🟩🟩🟩⬜⬜⬜🟩🟩🟩🟩🟩🟩🟩🟩🟩⬜🏠🟩🟩🟩🟩🔲   |   
🔲🟩🟩⬜🟩🟩🟩🟩🟩🟩⬜🟩🟩🟩🟩🟩⬜⬜🚒🟩🟩🟩🟩🟩🟩🟩🟩🟩⬜🏠🟩🟩🟩🟩🔲   |   
🔲🟩🟩⬜🟩🟩🟩🟩🟩🟩⬜🟩🟩🟩🟩🚒⬜⬜⬜🟩🟩🔥🟩🟩🟩🟩🟩🟩⬜🟩🟩🟩🟩🟩🔲   |   
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜🚑⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜   |   
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜   |   
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜🚑⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜   |   
🔲⬜🟩🟩⬜🟩🟩🟩⬜🟩🟩🟩🟩🟩🟩🟩⬜⬜⬜🚒🟩

In [4]:
import numpy as np
import time

GRID_SIZE = 35

SAFE_ZONE = -1
CIVILIAN = 1
DISASTER = 2
EMS_VEHICLE = 3
FIREFIGHTER = 6
ROAD = 4
BUILDING = 5
WATER = 7
EXTINGUISHED = 8


emoji_map = {
    SAFE_ZONE: "🔲",
    0: "🟩",
    CIVILIAN: "👥",
    DISASTER: "🔥",
    EMS_VEHICLE: "🚑",
    FIREFIGHTER: "🚒",
    ROAD: "⬜",
    BUILDING: "🏠",
    WATER: "🟦", 
    EXTINGUISHED: "⬜"
}

KEY_DESCRIPTION = {
    "🔲": "Safe Zone (Edge)",
    "🟩": "Grass",
    "👥": "Civilians needing rescue",
    "🔥": "Fire (spreading)",
    "🚑": "EMS Vehicle",
    "🚒": "Firefighter",
    "⬜": "Road / Fire fully out", 
    "🏠": "Houses",
    "🟦": "Fire being extinguished"
}




def show_grid_with_key(grid, step):
    if step % 5 == 0:
        grid_rows = ["".join(emoji_map[cell] for cell in row) for row in grid]
        key_rows = [f"{symbol} - {desc}" for symbol, desc in KEY_DESCRIPTION.items()]
        max_rows = max(len(grid_rows), len(key_rows))
        grid_rows += [""] * (max_rows - len(grid_rows))
        key_rows += [""] * (max_rows - len(key_rows))
        for grid_row, key_row in zip(grid_rows, key_rows):
            print(f"{grid_row}   |   {key_row}")
        print("\n")

# Initialize grid
grid = np.zeros((GRID_SIZE, GRID_SIZE))
grid[0, :] = SAFE_ZONE
grid[-1, :] = SAFE_ZONE
grid[:, 0] = SAFE_ZONE
grid[:, -1] = SAFE_ZONE

mid = GRID_SIZE // 2
road_width = 1
for i in range(-road_width, road_width + 1):
    grid[mid + i, :] = ROAD
    grid[:, mid + i] = ROAD

num_houses = 40
houses = []
for _ in range(num_houses):
    x, y = np.random.randint(1, GRID_SIZE - 3, size=2)
    if np.all(grid[x:x + 2, y:y + 2] == 0):
        grid[x:x + 2, y:y + 2] = BUILDING
        houses.append((x, y))

def connect_house_to_road(grid, house_x, house_y):
    road_positions = np.argwhere(grid == ROAD)
    best_road = min(road_positions, key=lambda r: abs(r[0] - house_x) + abs(r[1] - house_y))
    cx, cy = house_x, house_y
    while (cx, cy) != tuple(best_road):
        grid[cx, cy] = ROAD
        if abs(cx - best_road[0]) > abs(cy - best_road[1]):
            cx += np.sign(best_road[0] - cx)
        else:
            cy += np.sign(best_road[1] - cy)

for hx, hy in houses:
    connect_house_to_road(grid, hx, hy)

num_civilians = 15
civilians = []
for _ in range(num_civilians):
    house_x, house_y = houses[np.random.randint(len(houses))]
    offsets = [(-1, 0), (1, 0), (0, -1), (0, 1)]
    np.random.shuffle(offsets)
    for dx, dy in offsets:
        cx, cy = house_x + dx, house_y + dy
        if 1 <= cx < GRID_SIZE - 1 and 1 <= cy < GRID_SIZE - 1 and grid[cx, cy] == 0:
            grid[cx, cy] = CIVILIAN
            civilians.append((cx, cy))
            break

ems_units = [(mid - 1, mid), (mid + 1, mid)]
for ems_x, ems_y in ems_units:
    grid[ems_x, ems_y] = EMS_VEHICLE

firefighters = [(mid - 3, mid + 1), (mid + 3, mid - 1), (mid - 2, mid - 2), (mid + 2, mid + 2)]
for fx, fy in firefighters:
    grid[fx, fy] = FIREFIGHTER

disaster_x, disaster_y = np.random.randint(1, GRID_SIZE - 1, size=2)
grid[disaster_x, disaster_y] = DISASTER

def spread_fire(grid, step):
    new_grid = grid.copy()
    spread_chance = min(0.1 + (step * 0.05), 0.6)
    for x in range(1, GRID_SIZE - 1):
        for y in range(1, GRID_SIZE - 1):
            if grid[x, y] == DISASTER:
                for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                    if np.random.rand() < spread_chance and grid[x + dx, y + dy] in [0, ROAD, BUILDING]:
                        new_grid[x + dx, y + dy] = DISASTER
    return new_grid

def move_firefighters_toward_fire(grid, firefighters):
    fire_positions = np.argwhere(grid == DISASTER)
    new_firefighters = []
    for fx, fy in firefighters:
        if fire_positions.size == 0:
            new_firefighters.append((fx, fy))
            continue
        closest_fire = min(fire_positions, key=lambda f: abs(f[0] - fx) + abs(f[1] - fy))
        dx = np.sign(closest_fire[0] - fx)
        dy = np.sign(closest_fire[1] - fy)
        new_fx, new_fy = fx + dx, fy + dy
        if grid[new_fx, new_fy] in [0, ROAD]:
            grid[fx, fy] = 0
            grid[new_fx, new_fy] = FIREFIGHTER
            new_firefighters.append((new_fx, new_fy))
        else:
            new_firefighters.append((fx, fy))
    return grid, new_firefighters

def firefighters_extinguish(grid, firefighters):
    for fx, fy in firefighters:
        for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            nx, ny = fx + dx, fy + dy
            if 0 <= nx < GRID_SIZE and 0 <= ny < GRID_SIZE:
                if grid[nx, ny] == DISASTER:
                    grid[nx, ny] = WATER
    return grid

def spread_water(grid):
    new_grid = grid.copy()
    for x in range(1, GRID_SIZE - 1):
        for y in range(1, GRID_SIZE - 1):
            if grid[x, y] == WATER:
                for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                    nx, ny = x + dx, y + dy
                    if grid[nx, ny] == DISASTER:
                        new_grid[nx, ny] = WATER
    return new_grid

def remove_fire(grid):
    for x in range(GRID_SIZE):
        for y in range(GRID_SIZE):
            if grid[x, y] == WATER:
                grid[x, y] = EXTINGUISHED
            elif grid[x, y] == EXTINGUISHED:
                grid[x, y] = 0
    return grid

rescued_count = 0
max_rescues = 15
total_evacuated_time = 0
steps = 25
for step in range(steps):
    if rescued_count >= max_rescues:
        break
    grid = spread_fire(grid, step)
    grid, firefighters = move_firefighters_toward_fire(grid, firefighters)
    grid = firefighters_extinguish(grid, firefighters)
    grid = spread_water(grid)
    grid = remove_fire(grid)
    show_grid_with_key(grid, step)
    time.sleep(0.5)

print(f"\n✅ Simulation complete! {rescued_count} civilians rescued in {total_evacuated_time} minutes. ⏳🚑")


🔲🔲🔲🔲🔲🔲🔲🔲🔲🔲🔲🔲🔲🔲🔲🔲⬜⬜⬜🔲🔲🔲🔲🔲🔲🔲🔲🔲🔲🔲🔲🔲🔲🔲🔲   |   🔲 - Safe Zone (Edge)
🔲🟩🟩🟩⬜🏠🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩⬜⬜⬜🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩⬜🏠🟩🔲   |   🟩 - Grass
🔲🟩🟩🟩⬜🏠🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩⬜⬜⬜⬜⬜⬜🏠🟩🟩🟩🟩🟩🟩🟩🟩⬜🏠🟩🔲   |   👥 - Civilians needing rescue
🔲🟩🟩🟩⬜🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩⬜⬜⬜🟩🟩🏠🏠🟩🟩🔥🟩🟩🟩🟩⬜⬜🟩🟩🔲   |   🔥 - Fire (spreading)
🔲🟩🟩🟩⬜🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩⬜⬜⬜🟩🟩🟩🟩👥⬜🔥👥🟩⬜⬜⬜🏠🟩🟩🔲   |   🚑 - EMS Vehicle
🔲🟩🟩🟩⬜🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜🏠🏠🏠🟩🟩🔲   |   🚒 - Firefighter
🔲🟩🟩🟩⬜🟩🟩👥🟩🟩🟩🟩🟩🟩🟩🟩⬜⬜⬜🟩🟩🟩🟩🟩🟩🟩🏠🏠🏠🏠🟩🟩🟩🟩🔲   |   ⬜ - Road / Fire fully out
🔲🟩🟩🟩⬜⬜⬜⬜🏠🟩🟩🟩🟩🟩🟩🟩⬜⬜⬜🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🔲   |   🏠 - Houses
🔲🟩🟩⬜⬜🟩🟩🏠🏠🟩🟩🟩🟩👥🟩🟩⬜⬜⬜🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🔲   |   🟦 - Fire being extinguished
🔲🟩⬜⬜🟩🟩🟩🟩🟩🟩🟩🟩🟩⬜⬜⬜⬜⬜⬜🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🔲   |   
🔲👥⬜🏠🟩🟩🟩🟩🟩⬜🏠🟩🟩🏠🏠🟩⬜⬜⬜🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🔲   |   
🔲🟩⬜🏠🟩🟩🟩🟩🟩⬜🏠🟩🟩🟩🟩🟩⬜⬜⬜🟩🟩🟩🟩🟩🟩🟩👥⬜⬜⬜⬜⬜🏠🟩🔲   |   
🔲🟩⬜⬜⬜🏠🟩🟩🟩⬜🟩🟩🟩🟩🟩🟩⬜⬜⬜🟩🟩🟩🟩🟩🟩🟩🟩🏠🏠🟩🟩⬜🏠🟩🔲   |   
🔲🟩⬜🟩🏠🏠🟩🟩🟩⬜🟩🟩🟩🟩🟩🟩⬜⬜⬜🚒🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩⬜🟩🟩🔲   |   
🔲🟩⬜🟩🟩🟩🟩🟩🟩⬜⬜🏠🟩🟩⬜⬜🚒⬜🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩⬜🟩🟩🔲   |   
🔲🟩⬜🟩🟩🟩🟩🟩🟩⬜🏠🏠🟩🟩🏠🟩⬜⬜⬜🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩🟩⬜🟩🟩🔲   |   
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜🚑⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜   |   
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜   |   
⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜🚑⬜⬜🚒⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜⬜   |   
🔲⬜⬜🏠🟩🟩🟩🟩🟩⬜🟩🟩🟩🟩